## Import Libraries

In [ ]:
import numpy as np
import utils as ut
import models as mdl
import pandas as pd
import time
from datetime import datetime
import tensorflow as tf
import matplotlib.pyplot as plt
from IPython.display import clear_output
import logging
import warnings
warnings.filterwarnings("ignore")
warnings.filterwarnings( "ignore", module = "matplotlib\..*" )
logging.getLogger('matplotlib').setLevel(level=logging.CRITICAL)
logging.getLogger('tensorflow').setLevel(level=logging.FATAL)

from tqdm.auto import tqdm

import random,sys,importlib,keras,os,cv2,time,string,sklearn

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)


In [ ]:
# !pip install tensorflow==2.15 tensorflow_addons tensorflow_probability 


In [ ]:
import os
path_current  = os.getcwd()
if path_current.split('\\')[-1]=='E6_XAD':
    print(path_current)
    path_models = os.path.join(path_current,'models')
    if os.path.exists(path_models):
        path_models_VAE = os.path.join(path_models,'hazelnut_VAE_GAN_30000')
        if os.path.exists(path_models_VAE):
            print('-'*120)
            print(f'Model Found ::::::: {path_models_VAE}')
            print('-'*120)
        else:
            print('-'*120)
            print(f'------ Model NOT FOUND  ------ hazelnut_VAE_GAN_30000')
            print('-'*120)

In [ ]:
print(f"Tensorflow \t\t:\t{tf.__version__}")
# use_tfp= True
# if use_tfp:
    # import tensorflow_probability as tfp
    # print(f"TensorflowProb \t\t:\t{tfp.__version__}")
print(f"Python \t\t\t:\t{sys.version}")
print(f"Pandas \t\t\t:\t{pd.__version__}")
print(f"Numpy \t\t\t:\t{np.__version__}")
gpu = len(tf.config.list_physical_devices('GPU'))>0
print(f'GPU Usage\t\t:\t{gpu}')
if gpu:
    tf.tpu.XLAOptions(
    use_spmd_for_xla_partitioning=True, enable_xla_dynamic_padder=True)

## Path Setting

In [ ]:
submission_version = False
path_curr = os.getcwd()
path_notebook = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
path_supplementary = os.path.abspath(os.path.join(path_notebook, os.pardir))

if submission_version:
    DS_path_main = os.path.abspath(os.path.join(path_notebook,'dataset\mvtec_anomaly_detection'))
else:
    DS_path_main = 'D:\DS\AD\mvtec_anomaly_detection'

DS_name = 'hazelnut'
DS_path = os.path.join(DS_path_main,DS_name)

results_path        = os.path.join(path_curr,         'results')
path_csv            = os.path.join(results_path,          'csv')
models_main_path    = os.path.join(path_curr,          'models')
results_subs = os.path.join(results_path,        'test_results')

print('Found:', DS_path) if os.path.exists(DS_path) else print('Not Found:', DS_path)
print('Found:', models_main_path) if os.path.exists(models_main_path) else print('Not Found:', models_main_path)
print('Found:', results_path) if os.path.exists(results_path) else print('Not Found:', results_path)
print('Found:', path_csv) if os.path.exists(path_csv) else print('Not Found:', path_csv)

## Load Data

In [ ]:
# train_dir,gtruth_dir,test_dir = ut.Data.get_subdata(DS_path=DS_path)

train_dir = os.path.join(DS_path,'train')
gtruth_dir = os.path.join(DS_path,'ground_truth2')
test_dir = os.path.join(DS_path,'test2')


In [ ]:
##################   GENERAL CONFIGURATION     ##################
dpi=150

In [ ]:
importlib.reload(ut)   
codes_path = os.getcwd()
ut.Data.path_verifier(path=results_path)
ut.Data.path_verifier(path=results_path)
ut.Data.path_verifier(path=models_main_path)

In [ ]:
importlib.reload(ut)   
anomaly_types = sorted(os.listdir(test_dir))
anomaly_types_gt = sorted(os.listdir(gtruth_dir))
anomaly_types,anomaly_types_gt = ut.Data.get_anomaly_types(test_dir=test_dir,
                                                           dataset=DS_name,
                                                           gt_dir=gtruth_dir)
image_counter_cls = []
for i in range(len(anomaly_types)):
    image_counter_cls.extend([[anomaly_types[i],len(os.listdir(os.path.join(test_dir,anomaly_types[i])))]])

In [ ]:
# Loading Normal and Anamolous Data
importlib.reload(ut)

ds   = ut.Data.load_data_normal(DS_path=f'{train_dir}', batch_size=image_counter_cls[0][1])
print('************ Loading test data ************')
ds_a0 = ut.Data.load_data_abnormal(DS_path=f'{test_dir}/{anomaly_types[0]}', batch_size=image_counter_cls[0][1])
ds_a1 = ut.Data.load_data_abnormal(DS_path=f'{test_dir}/{anomaly_types[1]}', batch_size=image_counter_cls[1][1])
ds_a2 = ut.Data.load_data_abnormal(DS_path=f'{test_dir}/{anomaly_types[2]}', batch_size=image_counter_cls[2][1])
ds_a3 = ut.Data.load_data_abnormal(DS_path=f'{test_dir}/{anomaly_types[3]}', batch_size=image_counter_cls[3][1])
ds_a4 = ut.Data.load_data_abnormal(DS_path=f'{test_dir}/{anomaly_types[4]}', batch_size=image_counter_cls[4][1])

print('************ Loading GTruth ************')
gt_a1 = ut.Data.load_data_gtruth(DS_path=f'{gtruth_dir}/{anomaly_types[1]}', batch_size=image_counter_cls[1][1])
gt_a2 = ut.Data.load_data_gtruth(DS_path=f'{gtruth_dir}/{anomaly_types[2]}', batch_size=image_counter_cls[2][1])
gt_a3 = ut.Data.load_data_gtruth(DS_path=f'{gtruth_dir}/{anomaly_types[3]}', batch_size=image_counter_cls[3][1])
gt_a4 = ut.Data.load_data_gtruth(DS_path=f'{gtruth_dir}/{anomaly_types[4]}', batch_size=image_counter_cls[4][1])

ds_a = [ds_a0,ds_a1,ds_a2,ds_a3,ds_a4]
gt_a = [ds_a0,gt_a1,gt_a2,gt_a3,gt_a4]

In [ ]:
importlib.reload(ut)
importlib.reload(mdl)
augmentation_target = 'custom' # 'medium', 'full' minimal , custom
train_generator,test_generator, gtruth_generator= ut.Data2.load_data(train_dir = train_dir,
                                                                     test_dir = test_dir,
                                                                     gtruth_dir = gtruth_dir,
                                                                     dataset=DS_name,
                                                                     classes = anomaly_types,
                                                                     augmentation_target=augmentation_target,
                                                                    )

In [ ]:
image_counter = {class_name: len(os.listdir(os.path.join(test_dir, class_name))) for class_name in test_generator.class_indices}
print(image_counter)

min_image_count = min(image_counter.items(), key=lambda x: x[1])[1]
min_image_class = min(image_counter.items(), key=lambda x: x[1])[0]
print(min_image_class,min_image_count)
from collections import Counter
Counter(test_generator.classes)
lbls_cls = list(image_counter.keys())
count_cls = list(image_counter.values())
lbls_cls,count_cls

In [ ]:
x = train_generator.next()
print(x.shape)

fig, axes = plt.subplots(3, 5, figsize=(5,3))
for i, ax in enumerate(axes.flat):
    ax.imshow(x[i])
    ax.set_axis_off()
# plt.suptitle('Augmented training dataset')
plt.tight_layout(pad=0.20)
plt.show()

## VAE_GAN Model

In [ ]:
image_size=[128, 128]
latent_dim=32

In [ ]:
importlib.reload(ut)
importlib.reload(mdl)

latent_dim = 32
model_type = ['VAE_GAN']
mdl_type = model_type[0]
ittl = f'vae_{mdl_type}'

def build_model(image_size, latent_dim):
    encoder = mdl.Model_VAE_GAN_functions.get_encoder(latent_dim=latent_dim, print_summary=False)
    decoder = mdl.Model_VAE_GAN_functions.get_decoder(latent_dim=latent_dim, print_summary=False)
    discriminator = mdl.Model_VAE_GAN_functions.get_discriminator(shape=image_size + [3], print_summary=False)
    vae = mdl.VAE(encoder, decoder)
    model = mdl.VAE_GAN(vae, discriminator)
    return model
def save_model(model, prefix):
    model.save_weights(prefix+'_model', save_format='tf')
    model.vae.save_weights(prefix+'_vae', save_format='tf')
    model.vae.encoder.save_weights(prefix+'_encoder', save_format='tf')
    model.vae.decoder.save_weights(prefix+'_decoder', save_format='tf')
    model.discriminator.save_weights(prefix+'_discriminator', save_format='tf')
    print('Model saved.')
def load_model(model, prefix):
    model.load_weights(prefix+'_model')
    model.vae.load_weights(prefix+'_vae')
    model.vae.encoder.load_weights(prefix+'_encoder')
    model.vae.decoder.load_weights(prefix+'_decoder')
    model.discriminator.load_weights(prefix+'_discriminator')

In [ ]:
num_epochs = 30000
models_path = os.path.abspath(f'{models_main_path}/{DS_name}_VAE_GAN_{num_epochs}')
print('Model Path:',models_path)
model = build_model(image_size, latent_dim)
model.compile(optimizer=tf.keras.optimizers.Adam())
history_frame = pd.DataFrame(columns=['vae_loss', 'disc_loss', 'gen_los'])
prefix_fname = models_path + f'/{DS_name}_{mdl_type}_{num_epochs}'
history_fname = prefix_fname + '.csv'

In [ ]:
if os.path.exists(history_fname):
    print(f'Trying Loading file: {prefix_fname}')
    model.compile(optimizer=tf.keras.optimizers.Adam())
    load_model(model, prefix_fname)
    history_frame = pd.read_csv(history_fname, sep=',')
    print('*'*120)
    print(f'Model Found with \t:\t{len(history_frame)} epochs')
    epochs = {len(history_frame)}
    print('*'*120)
else:
    print(f'Model not Found \t:\t{history_fname}')

#### Train or Load Model

In [ ]:
train_model = False
###############################################################
save_weights = True
importlib.reload(ut)
importlib.reload(mdl)
ittl = f'vae_{mdl_type}'
train_batches = 5
for bb in range(train_batches):
    if train_model:
        start = time.time()
        history = model.fit(train_generator, 
                            epochs=len(history_frame) + num_epochs, 
                            initial_epoch=len(history_frame), 
                            verbose=1)
        history_frame = pd.concat([history_frame, pd.DataFrame(history.history)], ignore_index=True)
        print('Total training time:', time.time() - start)

        if save_weights:
            save_model(model, prefix_fname)
            history_frame.to_csv(history_fname, sep=',', index=False)
    else:
        if os.path.exists(history_fname):
            load_model(model, prefix_fname)
            print('*'*120)
            print(f'Model Loaded\t:\t {len(history_frame)}')
            print('*'*120)
            epochs = len(history_frame)
            break

In [ ]:
def running_mean(x, N):
    csum = np.cumsum(np.insert(x, 0, 0)) 
    return (csum[N:] - csum[:-N]) / float(N)

fig,axes = plt.subplots(3, 1, figsize=(10, 6))
for ii, metric in enumerate(['vae_loss', 'disc_loss', 'gen_los']):
    history_frame.loc[:, [metric]].plot(ax=axes[ii])
    axes[ii].plot(running_mean(history_frame[metric], 20))
axes[-1].set_xlabel('Epochs')
plt.tight_layout(pad=0.05)
# plt.savefig(f'{results_path}/{metric}_{mdl_type}_{epochs}_plot.png',dpi=200)
plt.show()

In [ ]:
images = np.array(list(train_generator.next()))
ind =20
_, b_img = model.predict(np.array([images[ind]]), verbose=False)
plt.subplot(131); plt.imshow(images[ind]); plt.xticks([]); plt.yticks([])
plt.subplot(132); plt.imshow(b_img[0]);    plt.xticks([]); plt.yticks([])

diff_img = images[ind]- b_img[0]
diff_img = np.linalg.norm(diff_img, axis=2)
diff_img = np.square(diff_img)
am_max = np.max(diff_img)

plt.subplot(133); plt.imshow(diff_img, cmap='bwr', vmin=-am_max, vmax=am_max); plt.xticks([]); plt.yticks([])
plt.show()

# Anomay Map from Test

In [ ]:
img_id = 2
a_type = anomaly_types[3]
print('Anomaly Type:\t',a_type)
print('Image No:\t',img_id)
I_A,gt = ut.Data.get_img_gt_by_index_class(anomaly_type=a_type,
                                            data_train=ds,
                                            data_test=ds_a,
                                            data_gtruth=gt_a,
                                            dataset=DS_name,
                                            image_no2=img_id)
zIN_mean, zIN_logvar, zIN = np.array(model.vae.encoder(np.array([I_A])))[:,0,:]
R_A = np.array(model.vae.decoder(np.array([zIN_mean]))[0])
plt.subplot(131);plt.imshow(I_A); plt.xticks([]); plt.yticks([]);
plt.subplot(132);plt.imshow(R_A); plt.xticks([]); plt.yticks([]);

anomaly_map = np.max(np.abs(R_A-I_A), axis=2)
am_max = np.max(anomaly_map)
plt.subplot(133); plt.imshow(anomaly_map, cmap='bwr', vmin=-am_max, vmax=am_max); plt.xticks([]); plt.yticks([])
plt.show()

<h2 style='color:red'> Find Anomaly Scores for Whole Test Set</h2>

In [ ]:
selector = 'max' # sum max
from tensorflow.keras.preprocessing import image_dataset_from_directory
test_ds = { at : image_dataset_from_directory(test_dir+'/'+at, labels=None, label_mode = None,
                                         image_size=image_size, interpolation='nearest', 
                                         batch_size=64, shuffle=False)
           for at in anomaly_types }
anomaly_scores = []
test_images = [ np.array(list(test_ds[at].take(1))[0]) for at in anomaly_types ]
for i, at in enumerate(anomaly_types):
    print(at, len(test_images[i]))
    z_mean, z_log_var, z = model.vae.encoder(test_images[i] / 255.0)
    reconstruction = model.vae.decoder(z_mean)
    if selector=='max':
        def get_anomaly_score(i, r):
            diff_norm = np.abs(r - i)
            return np.max(diff_norm)
    elif selector=='sum':
        def get_anomaly_score(i, r):
            diff_norm = np.abs(r - i)
            return np.sum(diff_norm)
    anomaly_scores.extend([(get_anomaly_score(reconstruction[j], test_images[i][j] / 255.0), 
                            at!='good', i, j,anomaly_types[i])  
                           for j in range(len(reconstruction))])
df_as = pd.DataFrame(anomaly_scores)
df_as.columns =['anomaly_score_new', 'good_or_bad','a_type_id','img_no','a_type' ]
print(np.min(df_as.anomaly_score_new),np.max(df_as.anomaly_score_new))
df_as.head(5)

In [ ]:
postfix_csv = f'{DS_name}_{epochs}_AS'
ascsv_filename = f'{path_csv}/testresults_{postfix_csv}.csv'
df_as.to_csv(ascsv_filename, sep=',')

<h2 style='color:red'> FIND Delta Opt</h2>

In [ ]:
importlib.reload(ut)
delta_opt = ut.Evaluate.find_optimal_separation_threshold(anomaly_scores)
print(f'delta_opt\t:\t{delta_opt}')

# Load_Image

In [ ]:
I_A = R_A = gt =y_true_flat=predicted_flat=ground_truth=binary_mask=None
def load_image(anomaly_type=None, img_id=None):
    global I_A,R_A,ground_truth,y_true_flat,predicted_flat,binary_mask
    I_A,ground_truth = ut.Data.get_img_gt_by_index_class(anomaly_type=anomaly_type,
                                               data_train=ds,
                                               data_test=ds_a,
                                               data_gtruth=gt_a,
                                               dataset=DS_name,
                                               image_no2=img_id)
    zIN_mean, zIN_logvar, zIN = np.array(model.vae.encoder(np.array([I_A])))[:,0,:]
    R_A = np.array(model.vae.decoder(np.array([zIN_mean]))[0])
    predicted_flat  = np.linalg.norm(I_A - R_A, axis=2).flatten()
    
    y_true_flat = (ground_truth[:,:,0]!=0).flatten()
    binary_mask = ground_truth[:,:,0].astype(np.uint8)
    del zIN_mean,zIN_logvar,zIN
    

In [ ]:
lbls_cls, count_cls

In [ ]:
a_class = lbls_cls[1]
img_id = 17

print('anomaly_class: \t',a_class)
print('Image no: \t',img_id)

load_image(anomaly_type=a_class, img_id=img_id)
plt.subplot(131);plt.imshow(I_A); plt.xticks([]);plt.yticks([]);
plt.subplot(132);plt.imshow(R_A); plt.xticks([]);plt.yticks([]);
plt.subplot(133);plt.imshow((ground_truth[:,:,0]), cmap='binary'); 
plt.xticks([]);plt.yticks([]);

In [ ]:
mask = mask_tp = None
def get_mask(mask_type='blend_2'):
    global mask,mask_tp
    if mask_type=='blend_2':
        mask = I_A*0.5+R_A*0.5
        mask_tp = f"x+x\'"
    elif mask_type=='blend_3':
            mask = I_A*0.33+R_A*0.33 + gt * 0.34
            mask_tp = f"x+x\'+gt"
    elif mask_type=='input':
            mask = I_A
            mask_tp = f"x+x\'+gt"
    elif mask_type=='reconstructed':
            mask = R_A
            mask_tp = f"x\'"
get_mask(mask_type='blend_2')

# XAD

## Import Libraries

In [ ]:
import shap
import shap_bpt
import matplotlib.pyplot as plt
from functools import partial

# LIME 
from lime.wrappers.scikit_image import SegmentationAlgorithm
from lime.lime_image import LimeImageExplainer
import skimage
from skimage.segmentation import quickshift
## 
from skimage.segmentation import mark_boundaries
#
from sklearn.metrics import roc_curve
from scipy.ndimage import gaussian_filter
from sklearn.metrics import roc_auc_score

plt.rc('text', usetex = True)

# Default Param

In [ ]:
num_explained_classes = 1
batch_size = 16
num_samples= 500
verbose=True

run_partial_test = False 
# run_partial_test = True
evaluate_explanation = True

# Functions

## Loss Function for XAI METHODS

In [ ]:
# From "General Frameworks for Anomaly Detection Explainability: A comparative study"
importlib.reload(ut)
from keras import backend as K
def mean_predict(model, data):
    z_mean, z_log_var, z = model.vae.encoder(data)
    reconstruction = model.vae.decoder(z_mean)
    return reconstruction
def lime_predict_loss(data, verbose=False):
    global model
    batch_size = K.shape(data)[0]
    reconstruction = mean_predict(model, data)
    MSE_loss = tf.reduce_sum(tf.square(reconstruction - data), axis=[1,2,3])
    return np.array([ [l] for l in MSE_loss ])

In [ ]:
score_functions     = [lime_predict_loss]

In [ ]:
img = np.array([I_A])
print('lime_predict_loss\t\t',lime_predict_loss(img)[0][0])

## Saliency_to_auc

In [ ]:
# def saliency_to_auc(heatmap, batch_size=4, method='del', num_samples=101, add_noise= True):
#     assert len(heatmap.shape)==2 and heatmap.dtype in (float, np.float64, np.float32)
#     if add_noise:
#         rng = np.random.default_rng(12345)
#         heatmap = heatmap + rng.normal(0.0, 0.000000001, size=heatmap.shape)
    
#     xs, ys, ms, masks, qs = [], [], [], [], []
#     #heatmap = gaussian(heatmap, 2.0)
#     for i in np.linspace(start=1.0, stop=0.0, num=num_samples):
#         if method=='del':
#             epsilon = (1 if i==0.0 else 0)
#             q = (np.quantile(heatmap, q=i) - epsilon)
#             m = heatmap <= q
#             nx = (1.0 - np.sum(m) / m.size)
#         elif method=='ins':
#             epsilon = (1 if i==1.0 else 0)
#             q = (np.quantile(heatmap, q=i) + epsilon)
#             m = heatmap >= q
#             nx = (np.sum(m) / m.size)
#         else:
#             raise Exception()
            
#         if len(xs)==0 or nx != xs[-1]:
#             xs.append(nx)
#             masks.append(m)
#             ms.append(np.sum(heatmap[m]))
#             qs.append(q)
#             if len(masks) >= batch_size:
#                 y = shap_bpt_masked_model(np.array(masks))[:, 0]
#                 ys.extend(y)
#                 masks = []

#     if len(masks) > 0:
#         y = shap_bpt_masked_model(np.array(masks))[:, 0]
#         ys.extend(y)
    
#     xs, ys = np.array(xs), np.array(ys)
#     auc_reg, auc_eff, auc_mse = 0.0, 0.0, 0.0
#     assert(len(xs) == len(ys))
#     # compute the area under the curve - use the rectangle method
#     for i in range(1, len(xs)):
#         delta_x = abs(xs[i] - xs[i-1])
#         if delta_x > 0:
#             auc_reg += abs(delta_x * 0.5*(ys[i-1] + ys[i])) # base  height
#             auc_eff += abs(delta_x * 0.5*(ys[i-1] + ys[i] - ms[i-1] - ms[i])) # base *  height
#             auc_mse += abs(delta_x * 0.5*(ys[i-1] + ys[i] - ms[i-1] - ms[i])**2) # base  height^2

#     # return xs, ys, ms, auc_reg, auc_eff, auc_mse
#     return {'xs':xs, 'ys':ys, 'ms':ms, 'qs':qs, 'auc_reg':auc_reg, 'auc_eff':auc_eff, 'auc_mse':auc_mse,'method':method}

In [ ]:
def combine_groundtruth_explanation(gtruth, heatmap, threshold):
    if gtruth.ndim == 3:
        gt = gtruth[:,:,0]>0
    else:
        gt = gtruth[:,:]>0
    ht = (heatmap >= threshold).astype(np.uint8)
    img = np.zeros(shape=list(heatmap.shape)+[3], dtype=np.uint8)
    img[:,:,0] = 255*(1-gt)
    img[:,:,1] = 255*(1-ht)
    img[:,:,2] = 255*(1-ht)
    return img
def calc_IoU_curve(y_true, y_pred, add_noise=True):
    
    assert isinstance(y_true, np.ndarray)
    assert isinstance(y_pred, np.ndarray)
    assert len(y_true.shape)==1 and len(y_pred.shape)==1 # assumes y_true and y_pred to be flattened arrays
    assert len(y_true)==len(y_pred)
    assert y_true.dtype==np.dtype('bool') and np.issubdtype(y_pred.dtype, np.floating)
    if add_noise:
        rng = np.random.default_rng(12345)
        y_pred = y_pred + rng.normal(0.0, 0.000000001, size=y_pred.shape)

    yd = np.array(sorted(zip(y_pred, y_true), reverse=True))
    X2   = np.zeros(len(y_pred))
    IoU2 = np.zeros(len(y_pred))
    Th   = np.zeros(len(y_pred))
    
    nT = np.sum(y_true)
    nInt = 0
    for i in range(len(y_pred)):
        if yd[i,1]: 
            nInt += 1
        
        IoU2[i] = nInt / (i + nT - nInt)
        X2[i] = i
        Th[i] = yd[i,0]
        
    X2 = X2 / len(y_pred)
    auc_IoU = 0
    for i in range(1, len(y_pred)):
        auc_IoU += (X2[i] - X2[i-1]) * (IoU2[i] + IoU2[i-1]) / 2.0
    
    best_pt = np.argmax(IoU2)
    
    if np.sum(y_pred) == 0:
        return X2, np.zeros_like(X2), Th[best_pt], X2[best_pt], 0
    else:
        return X2, IoU2, Th[best_pt], X2[best_pt], auc_IoU

In [ ]:
# def saliency_to_auc(f_masked,heatmap, f_S, f_0, predicted_cls, batch_size=4, method='del', num_samples=101, 
#                     rule='trapezoid'):

#     # print(f'Computing AUC with method={method}, batch_size={batch_size}, num_samples={num_samples}, rule={rule}')
#     # print('from saliency_to_auc',heatmap.shape, heatmap.dtype, heatmap.min(), heatmap.max())
#     assert isinstance(heatmap, np.ndarray)
#     assert len(heatmap.shape)==2 and np.issubdtype(heatmap.dtype, np.floating)

#     nu_max = max(f_S, f_0)
#     nu_min = min(f_S, f_0)

#     xs, ys, ms, masks, qs = [], [], [], [], []
#     for i, value in enumerate(np.linspace(start=1.0, stop=0.0, num=num_samples)):
#         if method=='del':
#             epsilon = (1 if value==0.0 else 0)
#             q = (np.quantile(heatmap, q=value) - epsilon)
#             m = heatmap <= q
#             nx = (1.0 - np.sum(m) / m.size)
#         elif method=='ins':
#             epsilon = (1 if value==1.0 else 0)
#             q = (np.quantile(heatmap, q=value) + epsilon)
#             m = heatmap >= q
#             nx = (np.sum(m) / m.size)
#         else:
#             raise Exception()
            
#         # add a new datapoint on the curve
#         if len(xs)==0 or nx != xs[-1]: 
#             assert m.dtype==bool and len(m.shape)==2
#             xs.append(nx)
#             masks.append(m)
#             ms.append(np.sum(heatmap[m]))
#             qs.append(q)

#         # evaluate the characteristic function
#         if len(masks) >= batch_size or (len(masks)>0 and i==(num_samples-1)):
#             y = f_masked(np.array(masks))[:, predicted_cls]
#             ys.extend(y)
#             masks = []

#     assert len(masks)==0    
#     xs, ys = np.array(xs), np.array(ys)
#     assert(len(xs) == len(ys))

#     # compute considering under/over shoots
#     overshoot_max = np.maximum(0, ys - nu_max) # overshoot for values exceeding the maximum
#     overshoot_min = np.maximum(0, nu_min - ys) # overshoot for values below the minimum
#     # adjust ys with the overshoot. Clip it inside the admitted range
#     y_adjusted = np.clip(ys - 2*overshoot_max + 2*overshoot_min, nu_min, nu_max)

#     # rescaling
#     ys_rescaled = (ys - nu_min) / (nu_max - nu_min)
#     y_adjusted_rescaled = (y_adjusted - nu_min) / (nu_max - nu_min)

#     auc, auc_r, auc_mae, auc_mse, auc_adj, auc_adjr = 0.0, 0.0, 0.0, 0.0, 0.0, 0.0

#     curve_range = range(1, len(xs)) if rule=='trapezoid' else range(len(xs))

#     # compute the area under the curve with the midpoint Riemann sum (i.e. the trapezoidal rule)
#     for i in curve_range:
#         if rule=='trapezoid':
#             delta_x = abs(xs[i] - xs[i-1])
#             assert delta_x > 0
#             y_mid   =         0.5*(ys[i-1] + ys[i])
#             y_r_mid =         0.5*(ys_rescaled[i-1] + ys_rescaled[i])
#             err_mid = y_mid - 0.5*(ms[i-1] - ms[i])
#             y_adj_mid =       0.5*(y_adjusted[i-1] + y_adjusted[i])
#             y_adjr_mid =      0.5*(y_adjusted_rescaled[i-1] + y_adjusted_rescaled[i])
#         else: # rectangles
#             delta_x = 1.0/num_samples if i==len(xs)-1 else abs(xs[i+1] - xs[i])
#             assert delta_x > 0
#             y_mid   =         ys[i]
#             y_r_mid =         ys_rescaled[i]
#             err_mid = y_mid - ms[i]
#             y_adj_mid =       y_adjusted[i]
#             y_adjr_mid =      y_adjusted_rescaled[i]


#         auc += abs(delta_x * y_mid) # base * height
#         auc_r += abs(delta_x * y_r_mid) # base * height
#         # auc_eff += abs(delta_x * err_mid) # base * height
#         auc_mae += abs(delta_x * err_mid) # base * height
#         auc_mse += abs(delta_x * (err_mid**2)) # base * height^2
#         auc_adj += abs(delta_x * y_adj_mid)
#         auc_adjr += abs(delta_x * y_adjr_mid)

#     return {'xs':xs, 'ys':ys, 'ms':ms, 'qs':qs, 'ysr':ys_rescaled,
#             'y_adj':y_adjusted, 'y_adjr':y_adjusted_rescaled, 
#             'method':method, #'class_id':class_id,
#             'auc':auc, 'auc_r':auc_r, #'auc_eff':auc_eff, 
#             'auc_mae':auc_mae, 'auc_mse':auc_mse, 'auc_rmse':np.sqrt(auc_mse), 
#             'auc_adj':auc_adj, 'auc_adjr':auc_adjr}

In [ ]:
def saliency_to_auc(nu, heatmap, f_S, f_0, predicted_cls, batch_size=4, method='del', num_samples=101, 
                    rule='trapezoid'):
    assert isinstance(heatmap, np.ndarray)
    assert len(heatmap.shape)==2 and np.issubdtype(heatmap.dtype, np.floating)

    # nu_max = max(f_S, f_0)
    # nu_min = min(f_S, f_0)

    xs, ys, ms, masks, qs = [], [], [], [], []
    for i, value in enumerate(np.linspace(start=1.0, stop=0.0, num=num_samples)):
        if method=='del':
            epsilon = (1 if value==0.0 else 0)
            q = (np.quantile(heatmap, q=value) - epsilon)
            m = heatmap <= q
            nx = (1.0 - np.sum(m) / m.size)
        elif method=='ins':
            epsilon = (1 if value==1.0 else 0)
            q = (np.quantile(heatmap, q=value) + epsilon)
            m = heatmap >= q
            nx = (np.sum(m) / m.size)
        else:
            raise Exception()
            
        # add a new datapoint on the curve
        if len(xs)==0 or nx != xs[-1]: 
            assert m.dtype==bool and len(m.shape)==2
            xs.append(nx)
            masks.append(m)
            ms.append(np.sum(heatmap[m]))
            qs.append(q)

        # evaluate the characteristic function
        if len(masks) >= batch_size or (len(masks)>0 and i==(num_samples-1)):
            y = nu(np.array(masks))[:, predicted_cls]
            ys.extend(y)
            masks = []

    assert len(masks)==0    
    xs, ys = np.array(xs), np.array(ys)
    assert(len(xs) == len(ys))

    # compute considering under/over shoots
    if f_S > f_0:
        overshoot_max = np.maximum(0, ys - f_S) # overshoot for values exceeding the maximum f(S)
        overshoot_min = np.maximum(0, f_0 - ys) # overshoot for values below the minimum f(0)
    else: # f(S) < f(0)
        overshoot_max = np.maximum(0, ys - f_0) # overshoot for values exceeding the maximum f(0)
        overshoot_min = np.maximum(0, f_S - ys) # overshoot for values below the minimum f(S)

    # clip ys, no oveshoots
    y_clipped = np.clip(ys, min(f_S, f_0), max(f_S, f_0))
    # adjust ys with the overshoot. Clip it inside the admitted range
    y_adjusted = np.clip(ys - 2*overshoot_max + 2*overshoot_min, min(f_S, f_0), max(f_S, f_0))

    # rebase to f(0)
    if f_S > f_0:
        flipped = False
        ys = ys - f_0 
        y_clipped = y_clipped - f_0 
        y_adjusted = y_adjusted - f_0
    else: # f(S) < f(0)
        flipped = True
        ys = f_0 - ys 
        y_clipped = f_0 - y_clipped 
        y_adjusted = f_0 - y_adjusted

    # rescaling
    ys_rescaled = ys / abs(f_S - f_0)
    y_clipped_rescaled = y_clipped / abs(f_S - f_0)
    y_adjusted_rescaled = y_adjusted / abs(f_S - f_0)

    auc, auc_r, auc_mae, auc_mse, auc_adj, auc_adjr, auc_clip, auc_clipr = 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0

    curve_range = range(1, len(xs)) if rule=='trapezoid' else range(len(xs))

    # compute the area under the curve with the midpoint Riemann sum (i.e. the trapezoidal rule)
    for i in curve_range:
        if rule=='trapezoid':
            delta_x = abs(xs[i] - xs[i-1])
            assert delta_x > 0
            y_mid   =         0.5*(ys[i-1] + ys[i])
            y_r_mid =         0.5*(ys_rescaled[i-1] + ys_rescaled[i])
            err_mid = y_mid - 0.5*(ms[i-1] - ms[i])
            y_clip_mid =       0.5*(y_clipped[i-1] + y_clipped[i])
            y_clipr_mid =      0.5*(y_clipped_rescaled[i-1] + y_clipped_rescaled[i])
            y_adj_mid =       0.5*(y_adjusted[i-1] + y_adjusted[i])
            y_adjr_mid =      0.5*(y_adjusted_rescaled[i-1] + y_adjusted_rescaled[i])
        else: # rectangles
            delta_x = 1.0/num_samples if i==len(xs)-1 else abs(xs[i+1] - xs[i])
            assert delta_x > 0
            y_mid   =         ys[i]
            y_r_mid =         ys_rescaled[i]
            err_mid = y_mid - ms[i]
            y_clip_mid =       y_clipped[i]
            y_clipr_mid =      y_clipped_rescaled[i]
            y_adj_mid =       y_adjusted[i]
            y_adjr_mid =      y_adjusted_rescaled[i]


        auc += abs(delta_x * y_mid) # base * height
        auc_r += abs(delta_x * y_r_mid) # base * height
        # auc_eff += abs(delta_x * err_mid) # base * height
        auc_mae += abs(delta_x * err_mid) # base * height
        auc_mse += abs(delta_x * (err_mid**2)) # base * height^2
        auc_clip += abs(delta_x * y_clip_mid)
        auc_clipr += abs(delta_x * y_clipr_mid)
        auc_adj += abs(delta_x * y_adj_mid)
        auc_adjr += abs(delta_x * y_adjr_mid)

    return {'xs':xs, 'ms':ms, 'qs':qs, 
            'f_0':f_0, 'f_S':f_S, 'flipped':flipped, 
            'ys':ys, 'ysr':ys_rescaled,
            'y_clip':y_clipped, 'y_clipr':y_clipped_rescaled, 
            'y_adj':y_adjusted, 'y_adjr':y_adjusted_rescaled, 
            'method':method, 'predicted_cls':predicted_cls,
            'auc':auc, 'auc_r':auc_r,
            'auc_mae':auc_mae, 'auc_mse':auc_mse, 'auc_rmse':np.sqrt(auc_mse), 
            'auc_clip':auc_clip, 'auc_clipr':auc_clipr,
            'auc_adj':auc_adj, 'auc_adjr':auc_adjr}

In [ ]:
def plot_feature_importance(feat_importance=None, plt=None):
    plt.figure(figsize=(2,2))
    vm = np.max(np.abs(feat_importance))
    plt.imshow(feat_importance, vmin = -vm, vmax = vm, cmap=shap_bpt.shapley_values_colormap)
    plt.xticks([]); plt.yticks([])
    plt.show()

In [ ]:
lime_predict_loss(np.expand_dims(I_A,axis=0))

# ShapBPT

## Shap_AA

In [ ]:
def shap_bpt_masked_model(masks):
    X = []
    for m in masks:
        m3 = m
        img = R_A.copy()
        img[m3] = I_A[m3]
        X.append(img)
    return np.array(lime_predict_loss(np.array(X)))

In [ ]:
def get_aa_heatmaps(num_samples=100,verbose=False):
    explainer = shap_bpt.Explainer(shap_bpt_masked_model, I_A, num_explained_classes=num_explained_classes, verbose=verbose)
    shap_values_aa = explainer.explain_instance(num_samples, method='AA', verbose_plot=False, 
                                                batch_size=batch_size)
    return shap_values_aa[0]

In [ ]:
if run_partial_test:
    heatmap_shap = get_aa_heatmaps(num_samples=num_samples,verbose=True)
    
    plot_feature_importance(feat_importance=heatmap_shap,plt=plt)
    AUCD = saliency_to_auc(heatmap_shap, method='del', batch_size=batch_size)
    AUCI = saliency_to_auc(heatmap_shap, method='ins', batch_size=batch_size)
    plt.figure(figsize=(4.5,2))
    plt.subplot(121);plt.plot(AUCD['xs'],AUCD['ys']);plt.xticks([]);plt.yticks([]);plt.title('del curve');
    plt.subplot(122);plt.plot(AUCI['xs'],AUCI['ys']);plt.xticks([]);plt.yticks([]);plt.title('ins curve');plt.show()

## Shap_BPT

In [ ]:
shap_bpt_explainer = shap_bpt.Explainer(shap_bpt_masked_model, I_A, num_explained_classes=num_explained_classes, verbose=verbose)

In [ ]:
def get_bpt_heatmaps(num_samples=100,verbose=False):
    bpt = shap_bpt.build_bpt_from_image((I_A*255).astype('uint8'))
    explainer = shap_bpt.Explainer(shap_bpt_masked_model, I_A, num_explained_classes=num_explained_classes, verbose=verbose)
    shap_values_bpt = explainer.explain_instance(num_samples, method='BPT',
                                             batch_size=batch_size,bpt=bpt)
    return shap_values_bpt[0]

In [ ]:
if run_partial_test:
    heatmap_shapbpt = get_bpt_heatmaps(num_samples=num_samples,verbose=True)
    
    plot_feature_importance(feat_importance=heatmap_shapbpt,plt=plt)
    # AUCD = {}
    AUCD = saliency_to_auc(heatmap_shapbpt, method='del', batch_size=batch_size)
    # AUCD = {}
    AUCI = saliency_to_auc(heatmap_shapbpt, method='ins', batch_size=batch_size)
    plt.figure(figsize=(4.5,2))
    plt.subplot(121);plt.plot(AUCD['xs'],AUCD['ys']);plt.xticks([]);plt.yticks([]); plt.title('del curve')
    plt.subplot(122);plt.plot(AUCI['xs'],AUCI['ys']);plt.xticks([]);plt.yticks([]); plt.title('ins curve');plt.show()

## LIME

In [ ]:
def get_segment_number(image, md):
    segments = quickshift(image, kernel_size=2, max_dist=md, ratio=0.95, rng=1234, sigma=0.05) 
    # segments = quickshift(image, kernel_size=2, max_dist=md, ratio=0.8, rng=1234, sigma=0.25) 
    return len(np.unique(segments)), segments

def search_segment_number(image, target_seg_no, init_max_dist=100):
    lmd, rmd = 0, init_max_dist
    lsn, _ = get_segment_number(image, lmd)
    rsn, _ = get_segment_number(image, rmd)
    niter = 0
    while niter<40 and rsn!=target_seg_no:
        niter += 1
        mmd = (lmd + rmd) / 2.0
        msn, _ = get_segment_number(image, mmd)
        if msn <= target_seg_no <= lsn:
            rsn, rmd = msn, mmd
        else:
            lsn, lmd = msn, mmd
    return get_segment_number(image, rmd) #rmd

In [ ]:
score_functions = [lime_predict_loss]
score_function =score_functions[0]
score_function_name = f'{score_function.__name__}'
score_function_name = score_function_name.replace('lime_predicter_', 'lm_p_')
score_function_name = score_function_name.replace('lime_predict_', 'lm_p_')
score_function_name

In [ ]:
beta = None
def format_lime_heatmaps(segments, expl):
    global predicted_fS, predicted_f0
    class_heatmaps = []
    heatmap = np.zeros_like(segments, dtype=np.float32)
    for segm, importance in expl.local_exp[expl.top_labels[0]]:
        heatmap[ segments==segm ] += importance 
    return np.array(heatmap)
def get_heatmaps_lime(num_samples=100,num_segments=100,verbose=False):
    global beta
    segs,segments = search_segment_number(mask,target_seg_no=num_segments)
    def segments_getter(img):
        return segments
    # score_function_name = f'{score_function.__name__}'
    # score_function_name = score_function_name.replace('lime_predicter_', 'lm_p_')
    # score_function_name = score_function_name.replace('lime_predict_', 'lm_p_')
    lime_explainer = LimeImageExplainer(random_state=1234)
    st = time.time()
    
    explanation = lime_explainer.explain_instance(I_A, score_functions[0],
                                                  labels = [a_type],
                                                  segmentation_fn=segments_getter,
                                                  hide_color=R_A,
                                                  batch_size=batch_size,
                                                  num_samples=num_samples,
                                                 progress_bar=verbose
                                                 )
    if isinstance(explanation, tuple):
        explanation = explanation[2]
    # beta = ut.Explanation.get_beta_from_expl(explanation)
    heatmap_lime = format_lime_heatmaps(segments, explanation)
    beta = ut.Explanation.get_beta_from_expl(explanation)
    return heatmap_lime

In [ ]:
if run_partial_test:
    heatmap_lime = get_heatmaps_lime(num_samples=num_samples, verbose=True)
    plot_feature_importance(feat_importance=heatmap_lime,plt=plt)
    AUCD = saliency_to_auc(heatmap_lime, method='del', batch_size=batch_size)
    AUCI = saliency_to_auc(heatmap_lime, method='ins', batch_size=batch_size)
    plt.figure(figsize=(4.5,2))
    plt.subplot(121);plt.plot(AUCD['xs'],AUCD['ys']);plt.xticks([]);plt.yticks([]);plt.title('del curve');
    plt.subplot(122);plt.plot(AUCI['xs'],AUCI['ys']);plt.xticks([]);plt.yticks([]);plt.title('ins curve');plt.show()

## PLOT OWEN VALUES

In [ ]:
if run_partial_test:
    shap_bpt_explainer = shap_bpt.Explainer(shap_bpt_masked_model, I_A, num_explained_classes=num_explained_classes, verbose=verbose)
    shap_bpt.plot_owen_values(shap_bpt_explainer, [np.expand_dims(heatmap_shapbpt,axis= 0),
                                                   np.expand_dims(heatmap_shap, axis= 0),
                                                   np.expand_dims(heatmap_lime,axis= 0)],
                              [a_type],
                              names=['ShapBPT','ShapAA','LIME'])

In [ ]:
if run_partial_test:
    plt.figure(figsize=(12,4))
    plt.subplot(161);plt.imshow(I_A);          plt.xticks([]);plt.yticks([]);
    plt.subplot(162);plt.imshow(R_A);          plt.xticks([]);plt.yticks([]);
    plt.subplot(163);plt.imshow(ground_truth);  plt.xticks([]);plt.yticks([]);
    plt.subplot(164);plt.imshow(heatmap_shapbpt, vmin=-np.max(np.abs(heatmap_shapbpt)), vmax=np.max(np.abs(heatmap_shapbpt)), cmap=shap_bpt.shapley_values_colormap); 
    plt.xticks([]);plt.yticks([]);
    plt.subplot(165);plt.imshow(heatmap_shap, vmin=-np.max(np.abs(heatmap_shap)), vmax=np.max(np.abs(heatmap_shap)), cmap=shap_bpt.shapley_values_colormap); 
    plt.xticks([]);plt.yticks([]);
    plt.subplot(166);plt.imshow(heatmap_lime, vmin=-np.max(np.abs(heatmap_lime)), vmax=np.max(np.abs(heatmap_lime)), cmap=shap_bpt.shapley_values_colormap); 
    plt.xticks([]);plt.yticks([]);

In [ ]:
anomaly_map = am_max = None
def get_anomaly_map_score():
    global anomaly_map,am_max
    anomaly_map = np.max(np.abs(R_A-I_A), axis=2)
    am_max = np.max(anomaly_map)

# Combining All Functions

In [ ]:
verbose = False

methods = [
    ('BPT-100',         'xkcd:light pink',      partial(get_bpt_heatmaps, num_samples=100,verbose=verbose)),
    ('BPT-500',         'xkcd:light pink',      partial(get_bpt_heatmaps, num_samples=500,verbose=verbose)),
    ('BPT-1000',         'xkcd:light pink',      partial(get_bpt_heatmaps, num_samples=1000,verbose=verbose)),

    ('AA-100',          'xkcd:bright blue',     partial(get_aa_heatmaps, num_samples=100, verbose=verbose)),
    ('AA-500',          'xkcd:bright blue',     partial(get_aa_heatmaps, num_samples=500, verbose=verbose)),
    ('AA-1000',         'xkcd:bright blue',     partial(get_aa_heatmaps, num_samples=1000, verbose=verbose)),
    
    ('LIME-100',        'xkcd:bright lime',     partial(get_heatmaps_lime, num_segments=20, num_samples=100,verbose=verbose)),
    ('LIME-500',        'xkcd:bright lime',     partial(get_heatmaps_lime, num_segments=100, num_samples=500,verbose=verbose)),
    ('LIME-1000',        'xkcd:bright lime',     partial(get_heatmaps_lime, num_segments=200, num_samples=1000,verbose=verbose)),
]
for n,_,_ in methods:
    print(n)

methods_ls = [x for x,_,_ in methods ]

## Functions for Plotting

In [ ]:
def plot_save_figure(img=None,type_groundtruth=False, destroy_fig=True,save_fig=True,fname=None,transparent=True,dpi=250):
    fig=plt.figure(figsize=(2,2))
    if type_groundtruth:
        # ground_truth[:,:,0]), cmap='binary'
        plt.imshow(img[:,:,0], cmap='binary')    
    else:
        plt.imshow(img)
    plt.xticks([]); plt.yticks([])
    if save_fig:
        plt.savefig(fname,dpi=dpi, transparent=transparent, bbox_inches = 'tight')
    if destroy_fig:
        plt.close(fig)
    plt.show()
###################################################################################
def draw_plot_gt(I_A,ground_truth):
    contoured_image = I_A.copy()
    contoured_image = ut.Data.draw_gt_contour(groundtruth_image=ground_truth,input_image=I_A)
    binary_mask = ground_truth[:,:,0].astype(np.uint8)
    contours, _ = cv2.findContours(binary_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cv2.drawContours(contoured_image, contours, -1, (255, 255, 0), 1)  # (0, 255, 255) corresponds to yellow, 2 is the thickness
    return contoured_image
###################################################################################
def draw_heatmaps(feat_import=None,ground_truth=None,save_fig=True,fname=None,figsize=(2,2),dpi=250, transparent=True,destroy_fig=True):
    fig=plt.figure(figsize=figsize)
    vmax = np.quantile(np.abs(feat_import), 0.99)
    plt.imshow(feat_import, cmap=shap_bpt.shapley_values_colormap, vmin=-vmax, vmax=vmax)
    if ground_truth is not None:
        marked_h = mark_boundaries(np.tile((255,255,255,0), (feat_import.shape[0],feat_import.shape[1],1)), ground_truth[:,:,0], 
                             mode='thick', color=(0,0,0,1))
        plt.imshow(marked_h)
    plt.xticks([]); plt.yticks([])
    if save_fig:
        plt.savefig(fname,dpi=dpi, transparent=transparent, bbox_inches = 'tight')
    if destroy_fig:
        plt.close(fig)
    plt.show()

###################################################################################
props = dict(boxstyle='round', facecolor='white', alpha=0.6, pad=0.2)
def draw_IoU(feat_import=None,IoU_=None,max_IoU_=None,save_fig=True,fname=None,figsize=(2,2),dpi=250,transparent=True, fontsize=18,destroy_fig=True):
    props = dict(boxstyle='round', facecolor='white', alpha=0.6, pad=0.2)
    fig=plt.figure(figsize=figsize)
    img = ut.visualize.vis_IoU(feat_import, IoU_,ground_truth[:,:,0]!=0)
    plt.imshow(img)
    plt.text(8,22, f"$${max_IoU_:.3}$$", fontsize=fontsize, bbox=props) #,weight='bold')
    plt.xticks([]); plt.yticks([])
    if save_fig:
        plt.savefig(fname,dpi=dpi, transparent=transparent, bbox_inches = 'tight')
    if destroy_fig:
        plt.close(fig)
    plt.show()
def draw_anomaly_map(img=None,save_fig=True,fname=None,figsize=(2,2),dpi=250,transparent=True,destroy_fig=True):
    fig=plt.figure(figsize=figsize)
    vm = np.max(np.abs(img))
    plt.imshow(img, cmap=shap_bpt.shapley_values_colormap, vmin = -vm, vmax = vm)
    plt.xticks([]); plt.yticks([])
    if save_fig:
        plt.savefig(fname,dpi=dpi, transparent=transparent, bbox_inches = 'tight')
    if destroy_fig:
        plt.close(fig)
    plt.show()

##################### FULl PLOTS
def plot_figures_full(plot_colorbar=False,plot_title=True,fontsize= 14):
    
    
    if plot_colorbar:
        fraction,shrink,pad      = 0.048,1.0,0.02
    fig,axes = plt.subplots(1, len(methods)+4, figsize=(15, 1.5))
    axes[0].imshow(I_A); axes[0].set_title('Input', fontsize=fontsize)
    axes[1].imshow(R_A); axes[1].set_title('Recons', fontsize=fontsize)
    vm = np.max(np.abs(anomaly_map))
    axes[2].imshow(anomaly_map, cmap=shap_bpt.shapley_values_colormap, vmin = -vm, vmax = vm)
    axes[2].set_title('Anomaly Map', fontsize=fontsize)
    contoured_image = draw_plot_gt(I_A,ground_truth)
    print(f'{"Method":<20} Sum(heatmaps)')
    print('-'*120)
    axes[-1].imshow(contoured_image); axes[-1].set_title('GroundTruth', fontsize=fontsize)
    for ii, (n,c,_) in enumerate(methods):
        ax = axes[ii+3]
        vmax = np.max(np.abs(heatmaps[n]))
        heatmap_clr_bar=ax.imshow(heatmaps[n], cmap=shap_bpt.shapley_values_colormap, vmin=-vmax, vmax=vmax)
        marked_h = mark_boundaries(np.tile((255,255,255,0), (heatmaps[n].shape[0],heatmaps[n].shape[1],1)), ground_truth[:,:,0], 
                             mode='thick', color=(0,0,0,1))
        ax.imshow(marked_h)
        if plot_colorbar:
            cbar = plt.colorbar(heatmap_clr_bar , pad=pad, orientation='horizontal',fraction=fraction, shrink=shrink,ax=ax)
        print(f'{n:<20} {np.sum(heatmaps[n][0])}')
        if plot_title:
            ax.set_title(f'{n}', fontsize=fontsize)
    for ax in axes:
        ax.set_xticks([]) ; ax.set_yticks([])
    plt.subplots_adjust()
    plt.tight_layout(pad = 0.05)
    plt.show()
def plot_evaluations_full(plot_add_legend = False,fontsize=14):
    fig,axes = plt.subplots(2, len(methods), figsize=(2*(len(methods)+2), 3))
    aucD,aucI= {},{}
    for ii, (n,c,_) in enumerate(methods):
        aucD[n] = saliency_to_auc(heatmaps[n], method='del', batch_size=batch_size)
        aucI[n] = saliency_to_auc(heatmaps[n], method='ins', batch_size=batch_size)
        
        axes[0][ii].plot(aucD[n]['xs'], aucD[n]['ys'], color='orange', label=f"{aucD[n]['method']}-{n}")
        axes[1][ii].plot(aucI[n]['xs'], aucI[n]['ys'], color='blue', label=f"{aucI[n]['method']}-{n}")
        if ii==0:
            yloc = np.max(aucD[n]['ys'])-10
            xloc = np.max(aucD[n]['xs'])-0.5
        else:
            yloc = np.max(aucD[n]['ys'])-10
            xloc = np.max(aucD[n]['xs'])-0.5
        
        if plot_add_legend:
            axes[0][ii].text(xloc,yloc, f"auc_reg:{aucD[n]['auc_reg']:0.3}\nauc_eff:{aucD[n]['auc_eff']:0.3}")
            axes[1][ii].text(np.max(aucI[n]['xs'])-0.5,np.max(aucI[n]['ys'])-10, f"auc_reg:{aucI[n]['auc_reg']:0.3}\nauc_eff:{aucI[n]['auc_eff']:0.3}")
        
        axes[0][ii].set_title(f'{n}',fontsize=fontsize)
    axes[0][0].set_ylabel(f"{aucD[n]['method']}",fontsize=fontsize)
    axes[1][0].set_ylabel(f"{aucI[n]['method']}",fontsize=fontsize)
    plt.show()
def plot_IoUs_full(plot_title=True,fontsize=14):
    props = dict(boxstyle='round', facecolor='white', alpha=0.6, pad=0.2)
    fig,axes = plt.subplots(1, len(methods)+4, figsize=(15, 1.5))
    axes[0].imshow(I_A); axes[0].set_title('Input', fontsize=fontsize)
    axes[1].imshow(R_A); axes[1].set_title('Recons', fontsize=fontsize)
    vm = np.max(np.abs(anomaly_map))
    axes[2].imshow(anomaly_map, cmap=shap_bpt.shapley_values_colormap, vmin = -vm, vmax = vm)
    axes[2].set_title('Anomaly Map', fontsize=fontsize)
    axes[-1].imshow(ground_truth); axes[-1].set_title('ground_truth', fontsize=fontsize)
    
    for ii, (n,_,_) in tqdm(enumerate(methods), desc='IoU',leave=False):
        ax = axes[ii+3]
        img, max_IoU, name = ut.visualize.vis_IoU(heatmaps[n], IoU[n]['max_IoU_old'],ground_truth[:,:,0]!=0),np.max(IoU[n]['Y']), f'{n}'
        ax.imshow(img)
        
        ax.text(10,25, f"IoU:{max_IoU:.3}", fontsize=fontsize, bbox=props) #,weight='bold')
        if plot_title:
            ax.set_title(f'{n}', fontsize=fontsize)
    for ax in axes:
        ax.set_xticks([]) ; ax.set_yticks([])
    plt.subplots_adjust()
    plt.tight_layout(pad = 0.05)
    

In [ ]:
def vis_IoU(shapley_values, threshold, ground_truth,verbose=False):
    

    pred = shapley_values.flatten() >= threshold
    real = ground_truth.flatten()
    image = np.full((len(pred), 3), 1.0, dtype=np.float32)
    if verbose:
        print(np.sum(pred), np.sum(real))
    image[ pred & real, : ]    = (0.0, 0.0, 0.75) # True Positives
    image[ pred & (~real), : ] = (1.0, 0.6, 0.2)  # False Positives
    image[ (~pred) & real, : ] = (1.0, 0.4, 1.0)  # False Negatives
    # plt.show()
    # plt.imshow(image.reshape(ground_truth.shape + (3,)))
    # plt.show()
    # print(np.min(shapley_values), np.min(image), np.max(image))
    # print('IoU:', np.sum(pred), np.sum(real), np.sum(pred & real), np.sum(pred & (~real)), np.sum((~pred) & real))
    return image.reshape(list(ground_truth.shape) + [3])

In [ ]:
props = dict(boxstyle='round', facecolor='white', alpha=0.6, pad=0.2)

def plot_figures_full_filtered(
    heatmaps, selected_methods=None, save_fig=True, plot_recon=False, fname=None, 
    figsize=(2, 2), dpi=250, plot_colorbar=False, plot_title=True, fontsize=14, 
    text_x=15, text_y=25,
    destroy_fig = True
):
    if plot_colorbar:
        fraction, shrink, pad = 0.048, 1.0, 0.02

    # Filter methods if selected_methods is provided
    filtered_methods = [m for m in methods if selected_methods is None or m[0] in selected_methods]

    # Number of columns depends on whether reconstruction is plotted
    num_cols = 3 + len(filtered_methods) * 2 if plot_recon else 2 + len(filtered_methods) * 2
    fig, axes = plt.subplots(1, num_cols, figsize=(15, 2))  # All in one row

    # First column: Input Image
    axes[0].imshow(I_A)
    if plot_title:
        axes[0].set_title('Input', fontsize=fontsize)

    # Second column: Reconstructed Image (if enabled)
    if plot_recon:
        axes[1].imshow(R_A)
        if plot_title:
            axes[1].set_title('Reconstruction', fontsize=fontsize)

    # Third column: Ground Truth
    gt_idx = 2 if plot_recon else 1  # Adjust index based on plot_recon flag
    axes[gt_idx].imshow(ground_truth[:, :, 0], cmap='binary')
    if plot_title:
        axes[gt_idx].set_title('Ground Truth', fontsize=fontsize)

    print(f'{"Method":<20} Sum(heatmaps)')
    print('-' * 120)

    # Start plotting from the correct index
    start_idx = gt_idx + 1  # Next column after Ground Truth

    for ii, (n, c, _) in enumerate(filtered_methods):
        # Column for Shapley values heatmap
        ax1 = axes[start_idx + ii * 2]  
        vmax = np.max(np.abs(heatmaps[n]))
        heatmap_clr_bar = ax1.imshow(heatmaps[n], cmap=shap_bpt.shapley_values_colormap, vmin=-vmax, vmax=vmax)

        # Overlay boundaries
        marked_h = mark_boundaries(
            np.tile((255, 255, 255, 0), (heatmaps[n].shape[0], heatmaps[n].shape[1], 1)),
            ground_truth[:, :, 0],
            color=(0, 0, 0, 1)
        )
        ax1.imshow(marked_h)

        if plot_colorbar:
            plt.colorbar(heatmap_clr_bar, pad=pad, orientation='horizontal', fraction=fraction, shrink=shrink, ax=ax1)

        print(f'{n:<20} {np.sum(heatmaps[n][0])}')

        if plot_title:
            ax1.set_title(f'Shapley: {n}', fontsize=fontsize)

        # Column for Max IoU Visualization (right next to the Shapley map)
        ax2 = axes[start_idx + ii * 2 + 1]

        # max___iou = IoU[n]['max_IoU_old']# X2[best_pt]
        max___iou = IoU[n][2]
        IoU__Y =  np.max(IoU[n][1])
        img, max_IoU, name = vis_IoU(heatmaps[n],max___iou , ground_truth[:, :, 0] != 0), IoU__Y, f'{n}'
        ax2.imshow(img)
        ax2.text(text_x, text_y, f"$${max_IoU:.3}$$", fontsize=fontsize, bbox=props)

    # Remove ticks from all axes
    for ax in axes:
        ax.set_xticks([])
        ax.set_yticks([])

    plt.subplots_adjust()
    plt.tight_layout(pad=0.05)
    plt.subplots_adjust(hspace=0.05, wspace=0.05)

    # Save figure if required
    if save_fig and fname:
        plt.savefig(fname, dpi=dpi, transparent=True, bbox_inches='tight')
    if destroy_fig:
        plt.close(fig)
    plt.show()


In [ ]:
# #############################################################################
# def fun_plot_performance(aucI,aucD,IoU,budget_set='100',ext_type='png',path=None,save_fig=True,fontsize=14):

#     fig,axes = plt.subplots(1,9, figsize=(20,2.5), sharex=True)#, sharey=True) #3,3  figsize=(8,2.2)
#     # if len(background_tensors)==1:
#     #     if params.model_type=='ideal':    
#     #         aucI_aa =  aucI['AA-100']
#     #         aucD_aa =  aucD['AA-100']
#     #         auc_IoU_aa =  IoU['AA-100']
#     #         ttl = 'PE'
#     #     else:
#     #         aucI_aa =  aucI['Partition-100']
#     #         aucD_aa =  aucD['Partition-100']
#     #         auc_IoU_aa =  IoU['Partition-100']
#     # else:
#     aucI_aa =  aucI['AA-100']
#     aucD_aa =  aucD['AA-100']
#     auc_IoU_aa =  IoU['AA-100']
#     ttl = 'AA'
#     aucI_bpt =  aucI['BPT-100']
#     aucD_bpt =  aucD['BPT-100']
#     auc_IoU_bpt =  IoU['BPT-100']
    
#     for i in range(9):
#         ax = axes.flat[i]
#         ############  AUC
#         if i==0: # insertion/regression
#             # Xa,Ya,Ma,La      = aucI_aa['xs'],aucI_aa['ys'],aucI_aa['ms'],aucI_aa['auc_reg']
#             title='$\\mathit{AUC}^{+}$'
#             Xa,Ya,Ma,La      = aucI_aa ['xs'],aucI_aa ['ys'],aucI_aa ['ms'],aucI_aa ['auc']
#             Xb,Yb,Mb,Lb      = aucI_bpt['xs'],aucI_bpt['ys'],aucI_bpt['ms'],aucI_bpt['auc']
#             # Xb,Yb,Mb,Lb = aucI_bpt[0], aucI_bpt[1], aucI_bpt[2], aucI_bpt[3]
#         elif i==1: # deletion/regression
#             title='$\\mathit{AUC}^{-}$'
#             Xa,Ya,Ma,La      = aucD_aa ['xs'],aucD_aa ['ys'],aucD_aa ['ms'],aucD_aa ['auc']
#             Xb,Yb,Mb,Lb      = aucD_bpt['xs'],aucD_bpt['ys'],aucD_bpt['ms'],aucD_bpt['auc']


#         ############  AUC_Adj
#         elif i==2: # insertion/error
#             title='$\\mathit{AUC-Adj}^{+}$'
#             Xa,Ya,Ma,La      = aucI_aa ['xs'],aucI_aa ['y_adj'],aucI_aa ['ms'],aucI_aa ['auc_adj']
#             Xb,Yb,Mb,Lb      = aucI_bpt['xs'],aucI_bpt['y_adj'],aucI_bpt['ms'],aucI_bpt['auc_adj']
#         elif i==3: # deletion/error
#             title='$\\mathit{AUC-Adj}^{-}$'
#             Xa,Ya,Ma,La      = aucD_aa ['xs'],aucD_aa ['y_adj'],aucD_aa ['ms'],aucD_aa ['auc_adj']
#             Xb,Yb,Mb,Lb      = aucD_bpt['xs'],aucD_bpt['y_adj'],aucD_bpt['ms'],aucD_bpt['auc_adj']



#         ############  AUC_r
#         elif i==4: # insertion/error
#             title='$\\mathit{AUC-r}^{+}$'
#             Xa,Ya,Ma,La      = aucI_aa ['xs'],aucI_aa ['ysr'],aucI_aa ['ms'],aucI_aa ['auc_r']
#             Xb,Yb,Mb,Lb      = aucI_bpt['xs'],aucI_bpt['ysr'],aucI_bpt['ms'],aucI_bpt['auc_r']            
        
#         elif i==5: # deletion/error
#             title='$\\mathit{AUC-r}^{-}$'
#             Xa,Ya,Ma,La      = aucD_aa ['xs'],aucD_aa ['ysr'],aucD_aa ['ms'],aucD_aa ['auc_r']
#             Xb,Yb,Mb,Lb      = aucD_bpt['xs'],aucD_bpt['ysr'],aucD_bpt['ms'],aucD_bpt['auc_r']

        

#         ############  AUC_Adj_r


#         elif i==6: # insertion/error
#             title='$\\mathit{AUC-Adj-r}^{+}$'
#             Xa,Ya,Ma,La      = aucI_aa ['xs'],aucI_aa ['y_adjr'],aucI_aa ['ms'],aucI_aa ['auc_adjr']
#             Xb,Yb,Mb,Lb      = aucI_bpt['xs'],aucI_bpt['y_adjr'],aucI_bpt['ms'],aucI_bpt['auc_adjr']
#         elif i==7: # deletion/error
#             title='$\\mathit{AUC-Adj-r}^{-}$'
#             Xa,Ya,Ma,La      = aucD_aa ['xs'],aucD_aa ['y_adjr'],aucD_aa ['ms'],aucD_aa ['auc_adjr']
#             Xb,Yb,Mb,Lb      = aucD_bpt['xs'],aucD_bpt['y_adjr'],aucD_bpt['ms'],aucD_bpt['auc_adjr']
            
#         elif i==8: # IoU
#             title='$\\mathit{AU}{\!-\!}\\mathit{IoU}$'
#             Xa,Ya,Ma,La = auc_IoU_aa[0], auc_IoU_aa[1], None, auc_IoU_aa[4]
#             Xb,Yb,Mb,Lb = auc_IoU_bpt[0], auc_IoU_bpt[1], None, auc_IoU_bpt[4]
#             xma, yma = auc_IoU_aa[3], np.max(auc_IoU_aa[1])
#             xmb, ymb = auc_IoU_bpt[3], np.max(auc_IoU_bpt[1])
            
#         # print(i,'  len(Xb): ',len(Xb),'  len(Yb): ', len(Yb), len(Xb)==len(Yb), '  len(Xa): ',len(Xa),'  len(Ya): ', len(Ya), len(Xa)==len(Ya))
#     #     ymax = max(np.max(Ya), np.max(Yb))
        
#         Sa, Sb = ('\\textbf', '') if La<Lb else ('', '\\textbf')
#         if i in [0,2,4,6, 8]:   Sa,Sb=Sb,Sa
#         ax.plot(Xa, Ya, c='#2465a6', label=f'{Sa}{{AA}} {La:.4}')
#         ax.fill_between(Xa, Ya, color='#2465a6', alpha=0.15, hatch='///')
#         # ax.scatter(Xa, Ya, c='black', s=5)
#         ax.plot(Xb, Yb, c='#9d2f4d', label=f'{Sb}{{BPT}} {Lb:.4}', alpha=0.80)
#         ax.fill_between(Xb, Yb, color='#9d2f4d', alpha=0.15, hatch='\\\\\\')
        
#         if i==8:
#             ax.scatter(xma, yma, s=40, color='blue')
#             ax.scatter(xmb, ymb, s=40, color='red')

#         if i < 4:
#             ax.axhline(f_S, ls='--', c='grey', zorder=0)
            
#         ax.axhline(0, c='lightgrey', zorder=0)
#         ax.legend(borderpad=0.2, labelspacing=0.1, loc='upper right' if i>=1 else 'lower right')#, bbox_to_anchor=(1,0))
#         ax.set_title(title, fontsize=fontsize)
    
#     # axes[0].set_xlabel('\% pixels inserted', fontsize=14)
#     # axes[1].set_xlabel('\% pixels deleted', fontsize=14)
#     # axes[2].set_xlabel('\% pixels inserted', fontsize=14)
#     # axes[3].set_xlabel('\% pixels deleted', fontsize=14)
    
#     plt.tight_layout()
#     if save_fig:
#         if path is not None:
#             plt.savefig(f'{path}_five_metrics_2.{ext_type}', dpi=150, bbox_inches='tight', pad_inches=0.02)
#         else:
#             print('path is None')
#     # plt.savefig(f'{path}/five_metrics_{background_type}_2.svg', dpi=150, bbox_inches='tight', pad_inches=0.02)
    
#     plt.show()

In [ ]:
from matplotlib import rc
rc('text',usetex=True)
rc('text.latex', preamble='\\usepackage{color}')

def fun_plot_performance(aucI, aucD, IoU,
                         list_variants=['BPT-100', 'AA-100', 'LIME-100'],
                         budget_set='100',
                         path=None,
                         file_name='',
                         save_fig=True,
                         fontsize=14,
                         set_title=False,
                         plot_lime=False,
                         ttl=None,
                         layout='row'  # 'row' or '2rows'
                         ):
    import numpy as np
    import matplotlib.pyplot as plt

    # Order of plots (grouped by column for 2-row layout)
    plot_defs = [
        {'title': '$\\mathit{AUC}^{+}$',         'typee': 'auc',        'from': 'aucI'}, # 0
        {'title': '$\\mathit{AUC}^{-}$',         'typee': 'auc',        'from': 'aucD'}, # 1
        {'title': '$\\mathit{AUC}^{+}-clip$',     'typee': 'auc_clip',   'from': 'aucI'}, # 2
        {'title': '$\\mathit{AUC}^{-}-clip$',     'typee': 'auc_clip',   'from': 'aucD'}, # 3
        {'title': '$\\mathit{AUC^{+}-Adj}$',     'typee': 'auc_adj',    'from': 'aucI'}, # 4
        {'title': '$\\mathit{AUC^{-}-Adj}$',     'typee': 'auc_adj',    'from': 'aucD'}, # 5
        
        {'title': '$\\mathit{AU}{\!-\!}\\mathit{IoU}$', 'typee': 'auc_iou', 'from': 'iou'}, # 6 
        #####################################################################

        {'title': '$\\mathit{AUC}^{+}-r$',       'typee': 'auc_r',      'from': 'aucI'}, # 7
        {'title': '$\\mathit{AUC}^{-}-r$',       'typee': 'auc_r',      'from': 'aucD'}, # 8
        {'title': '$\\mathit{AUC}^{+}-clip-r$', 'typee': 'auc_clip_r', 'from': 'aucI'}, # 9
        {'title': '$\\mathit{AUC}^{-}-clip-r$', 'typee': 'auc_clip_r', 'from': 'aucD'}, # 10
        {'title': '$\\mathit{AUC^{+}-Adj-r}$',   'typee': 'auc_adj_r',  'from': 'aucI'}, # 11
        {'title': '$\\mathit{AUC^{-}-Adj-r}$',   'typee': 'auc_adj_r',  'from': 'aucD'}, # 12 
        
    ]
    
    # Reorder plot_defs for 2-row layout: interleaved columns
    if layout == '2rows':
        top = plot_defs[0::2]
        bottom = plot_defs[1::2]
        plot_defs = [None]*(len(top)+len(bottom))
        plot_defs[::2] = top
        plot_defs[1::2] = bottom

    total_variant = len(plot_defs)

    # Layout config
    if layout == 'row':
        nrows, ncols = 1, total_variant
        fig_width, fig_height = 25, 2.5
    elif layout == '2rows':
        nrows, ncols = 2, int(np.ceil(total_variant / 2))
        fig_width, fig_height = 16, 5
    else:
        raise ValueError("layout must be either 'row' or '2rows'")

    fig, axes = plt.subplots(nrows, ncols, figsize=(fig_width, fig_height))
    axes = axes.flatten()

    # Alias
    aucI_aa = aucI[list_variants[0]]
    aucD_aa = aucD[list_variants[0]]
    auc_IoU_aa = IoU[list_variants[0]]

    aucI_bpt = aucI[list_variants[1]]
    aucD_bpt = aucD[list_variants[1]]
    auc_IoU_bpt = IoU[list_variants[1]]

    aucI_lime = aucI[list_variants[2]]
    aucD_lime = aucD[list_variants[2]]
    auc_IoU_lime = IoU[list_variants[2]]

    def get_params_auc(auc_, typee):
        return {
            'auc':        (auc_['xs'], auc_['ys'],     auc_['ms'], auc_['auc']),
            'auc_r':      (auc_['xs'], auc_['ysr'],    auc_['ms'], auc_['auc_r']),
            'auc_adj':    (auc_['xs'], auc_['y_adj'],  auc_['ms'], auc_['auc_adj']),
            'auc_adj_r':  (auc_['xs'], auc_['y_adjr'], auc_['ms'], auc_['auc_adjr']),
            'auc_clip':   (auc_['xs'], auc_['y_clip'], auc_['ms'], auc_['auc_clip']),
            'auc_clip_r': (auc_['xs'], auc_['y_clipr'],auc_['ms'], auc_['auc_clipr']),
        }.get(typee, (None, None, None, None))

    def get_params_iou(iou_, typee):
        if typee == 'auc_iou':

            # xmb, ymb = iou_[3], np.max(iou_[1])


            Xa, Ya, Ma, La = iou_[0], iou_[1], None, iou_[4]
            xma, yma = iou_[3], np.max(iou_[1])
            return Xa, Ya, Ma, La, xma, yma
        return None, None, None, None, None, None

    for i, config in enumerate(plot_defs):
        ax = axes[i]
        title = config['title']
        typee = config['typee']
        source = config['from']

        # Select correct dict
        if source == 'aucI':
            Xa, Ya, Ma, La = get_params_auc(aucI_aa, typee)
            Xb, Yb, Mb, Lb = get_params_auc(aucI_bpt, typee)
            Xl, Yl, Ml, Ll = get_params_auc(aucI_lime, typee)
        elif source == 'aucD':
            Xa, Ya, Ma, La = get_params_auc(aucD_aa, typee)
            Xb, Yb, Mb, Lb = get_params_auc(aucD_bpt, typee)
            Xl, Yl, Ml, Ll = get_params_auc(aucD_lime, typee)
        elif source == 'iou':
            Xa, Ya, Ma, La, xma, yma = get_params_iou(auc_IoU_aa,  typee)
            Xb, Yb, Mb, Lb, xmb, ymb = get_params_iou(auc_IoU_bpt, typee)
            Xl, Yl, Ml, Ll, xml, yml = get_params_iou(auc_IoU_lime,   typee)

        # Boldness logic
        Sa, Sb = ('\\textbf', '') if La < Lb else ('', '\\textbf')
        if i in [0,2,4,6,8]: Sa, Sb = Sb, Sa

        # Plot
        ax.plot(Xa, Ya, c='#2465a6', label=f'{Sa}{{AA}} {La:.4f}')
        ax.fill_between(Xa, Ya, color='#2465a6', alpha=0.15, hatch='///')
        ##########
        ax.plot(Xb, Yb, c='#9d2f4d', label=f'{Sb}{{BPT}} {Lb:.4f}', alpha=0.80)
        ax.fill_between(Xb, Yb, color='#9d2f4d', alpha=0.15, hatch='\\\\\\')
        ##########
        if plot_lime:
            ax.plot(Xl, Yl, c="#01B0A7", label=f'{Sb}{{LM}} {Ll:.4f}', alpha=0.80)
            ax.fill_between(Xl, Yl, color='#01B0A7', alpha=0.15, hatch='\\\\\\')


        if typee == 'auc_iou':
            ax.scatter(xma, yma, s=40, color='blue')
            ax.scatter(xmb, ymb, s=40, color='red')

        if i < 4:
            ax.axhline(0.0, ls='--', c='grey', zorder=0)  # placeholder for f_S - f_0

        ax.axhline(0, c='lightgrey', zorder=0)
        ax.legend(borderpad=0.2, labelspacing=0.1, loc='upper right' if i>=1 else 'lower right')
        ax.set_title(title, fontsize=fontsize)
    axes[-1].imshow(I_A); axes[-1].set_xticks([]); axes[-1].set_yticks([])
    axes[-1].set_title(ttl.split(",")[0])
    if set_title:
        plt.suptitle(f'{ttl}', fontsize=16)
    plt.tight_layout()

    if save_fig and path is not None:
        plt.savefig(f'{path}/{file_name}_paired_auc_metrics.png', dpi=150, bbox_inches='tight', pad_inches=0.02)
    plt.show()


# Save Plots for Figure 11

In [ ]:
# plot_selected_results = False
plot_selected_results = True

In [ ]:
# selected_methods=['AA-500', 'BPT-500','LIME-500']
selected_methods = methods_ls
selected_methods

In [ ]:
exp_type = 'full' if len(selected_methods)==len(methods_ls) else 'selected'
exp_type


In [ ]:
# f_0 = shap_bpt_masked_model(np.zeros((1,128,128), dtype=bool))[0,0]
# f_S = shap_bpt_masked_model(np.ones((1,128,128), dtype=bool))[0,0]
# print(f_S, f_0)

In [ ]:
destroy_fig         = True
save_ext            = 'svg'
plot_single_files   = False

# a_type_selected_ls = ['crack','cut','hole','print']
# img_id_selected_ls = [3,2,1]

a_type_selected_ls = ['crack']
img_id_selected_ls = [3]

font_size = 24 if exp_type == 'selected' else 14
if plot_selected_results:
    results_path_paper_figures = os.path.join(results_path, 'selected')
    os.makedirs(results_path_paper_figures,              exist_ok=True)

if plot_selected_results:
    for (a_type,img_id) in zip(a_type_selected_ls,img_id_selected_ls):
        print('Anomaly Type: \t',a_type,' Image: \t',img_id)
        load_image(anomaly_type=a_type, img_id=img_id)
        f_0 = shap_bpt_masked_model(np.zeros((1,128,128), dtype=bool))[0,0]
        f_S = shap_bpt_masked_model(np.ones((1,128,128), dtype=bool))[0,0]
        get_anomaly_map_score()
        ######################################################
        heatmaps = {}
        for n,_,funct in tqdm(methods, desc= 'Explanation'):
            if n not in selected_methods:
                continue
            heatmaps[n] = funct()
        ######################################################
        aucD,aucI,IoU = {},{},{}
        for n,_,funct in tqdm(methods, desc= 'Evaluation'): 
            if n not in selected_methods:
                continue
            aucD[n] = saliency_to_auc(shap_bpt_masked_model,heatmaps[n],f_S, f_0, predicted_cls=0, method='del', batch_size=batch_size)
            aucI[n] = saliency_to_auc(shap_bpt_masked_model,heatmaps[n],f_S, f_0, predicted_cls=0, method='ins', batch_size=batch_size)
            # IoU[n]  = calc_IoU_curve(ground_truth_final.flatten(), heatmaps[n].flatten())
            IoU[n]  = calc_IoU_curve(y_true_flat, heatmaps[n].flatten())
        ######################################################
        fname=f'{results_path_paper_figures}/{a_type}_{img_id}_{exp_type}.{save_ext}'
        
        if exp_type == 'selected':
            plot_figures_full_filtered(heatmaps,selected_methods=selected_methods,plot_recon=False,fname=fname,plot_colorbar=False,plot_title=False,
                                  fontsize= font_size,text_x=15 ,text_y=40, destroy_fig=False)
        else:
            plot_figures_full_filtered(heatmaps,selected_methods=selected_methods,plot_recon=True,fname=fname,plot_colorbar=False,plot_title=False,
                                  fontsize= font_size,text_x=15 ,text_y=40, destroy_fig=False)

In [ ]:
print(shap_bpt_masked_model(np.zeros((1,128,128), dtype=bool))[0,0])
print(shap_bpt_masked_model(np.ones((1,128,128), dtype=bool))[0,0])

shap_bpt_masked_model(np.expand_dims(ground_truth[0], axis=0))
# np.expand_dims(ground_truth[0], axis=0).shape

predicted_fG = shap_bpt_masked_model(np.expand_dims(ground_truth[0], axis=0))[0]
f_G = float(predicted_fG[0])


In [ ]:
def get_bg_values(f_masked,ground_truth,predicted_cls=0, verbose=False):
    # global f_G, f_B
    # evaluate the ground truth mask with the background replacement strategy for masking function
    predicted_fG = f_masked(np.expand_dims(ground_truth[0], axis=0))[0]
    f_G = float(predicted_fG[predicted_cls])
    # print(class_names[predicted_cls], f_G, predicted_cls, f_G)
    # print('softmax prob:', np_softmax(predicted_fG)[predicted_cls])

    # evaluate the backgrounf (negative of the ground truth mask)
    background_mask = np.logical_not(ground_truth)
    predicted_fB = f_masked(np.expand_dims(background_mask, axis=0))[0]
    f_B = float(predicted_fB[predicted_cls])
    # print(class_names[predicted_cls], f_B, predicted_cls, f_B)
    # print('softmax prob:', np_softmax(predicted_fB)[predicted_cls])

    # print()
    if verbose:
        print('nu(S):  ', round(f_S, 4))
        print('nu(G):  ', round(f_G, 4))
        print('nu(S/G):', round(f_B, 4))
        print('nu(0):  ', round(f_0, 4))
    return f_G,f_B
# f_G,f_B = get_bg_values(shap_bpt_masked_model,ground_truth)

In [ ]:
plot_figures_full_filtered(heatmaps,selected_methods=selected_methods,plot_recon=False,fname=fname,plot_colorbar=False,plot_title=False,
                                  fontsize= font_size,text_x=15 ,text_y=40, destroy_fig= False)

In [ ]:
# fun_plot_performance(aucI,aucD,IoU,path=f'{results_path_paper_figures}/{a_type}_{img_id}_{exp_type}')

In [ ]:
fun_plot_performance(aucI,aucD,IoU,path=results_path, file_name=img_id,layout='2rows',
                                     set_title=True,ttl=f'a_type: {a_type}, img_id: {img_id} exp_type: {exp_type}')

## RUN SELECTED ON ALL IMAGES to Save SVG

In [ ]:
ground_truth,ground_truths = None,None
union_ground_truths = []
ground_truths = []
brown_hair,black_hair,blond_hair = None,None,None
Eyeglasses = None
def load_ground_truth(intrested_class_lss,img_id=None, verbose=False):
    global ground_truth,ground_truths,union_ground_truth,union_ground_truths
    global brown_hair,Eyeglasses,anno_path

    if verbose:
        print('LAODING GT for ', intrested_class_lss)
    if att_csv_celeba:
        image_id = f'{img_id+1:06}.jpg'
    else:
        image_id = f'{img_id}.jpg'
    union_ground_truths = {}
    ground_truths = []
    for mask_1 in intrested_class_lss:
        ground_truths = []
        if mask_1 == 'Smiling':
            sub_masks = ['l_lip', 'mouth', 'u_lip']
        elif mask_1=='Eyeglasses':
            sub_masks = ['eye_g']
            Eyeglasses = df_attr[df_attr.image_id==image_id]['Eyeglasses'].values[0]==1
        #Black_Hair, Blond_Hair,Brown_Hair,Gray_Hair,Straight_Hair,Wavy_Hair
        elif mask_1=='Brown_Hair':
            brown_hair = df_attr[df_attr.image_id==image_id]['Brown_Hair'].values[0]==1
            sub_masks = ['hair']
        elif mask_1=='Black_Hair':
            black_hair = df_attr[df_attr.image_id==image_id]['Black_Hair'].values[0]==1
            sub_masks = ['hair']
        elif mask_1=='Blond_Hair':
            blond_hair = df_attr[df_attr.image_id==image_id]['Blond_Hair'].values[0]==1
            sub_masks = ['hair']
        elif mask_1=='Gray_Hair':
            brown_hair = df_attr[df_attr.image_id==image_id]['Gray_Hair'].values[0]==1
            sub_masks = ['hair']
        elif mask_1=='Straight_Hair':
            brown_hair = df_attr[df_attr.image_id==image_id]['Straight_Hair'].values[0]==1
            sub_masks = ['hair']
        elif mask_1=='Wavy_Hair':
            brown_hair = df_attr[df_attr.image_id==image_id]['Wavy_Hair'].values[0]==1
            sub_masks = ['hair']
               
        else:
            sub_masks = [mask_1]
        
        # Loop over each sub-mask
        for sub_mas in sub_masks:
            
            folder = find_folder(img_id, anno_pth)
            anno_subpath = os.path.join(ds_anno_path,str(folder))
            anno_filename = f'{os.path.join(anno_subpath, f"{img_id:05}_{sub_mas}")}.png'
            #print('sub_masks:',sub_masks,anno_filename,os.path.exists(anno_filename))
            # print(anno_filename,img_id,image_id,os.path.exists(anno_filename), brown_hair, Eyeglasses, df_attr[df_attr.image_id==image_id]['Eyeglasses'])
            #if not os.path.exists(anno_filename) or not brown_hair or not Eyeglasses and load_positives_only:
            if not os.path.exists(anno_filename):
                ground_truth = np.zeros((224,224)).astype(int)
            else:
                #print('anno_subpath: \t',anno_filename,img_id,mask_1)
                ground_truth = cv2.imread(anno_filename, cv2.IMREAD_COLOR)[:, :, ::-1]
                ground_truth = cv2.resize(ground_truth, [224, 224], interpolation=cv2.INTER_NEAREST)
                ground_truth = ground_truth[:,:,0].astype(int)
            ground_truths.append(ground_truth)
        
        # Calculate the union of the ground truths for the current mask
        if len(ground_truths) > 0:
            union_ground_truth = ground_truths[0]
            for ground_truth in ground_truths[1:]:
                union_ground_truth = cv2.bitwise_or(union_ground_truth, ground_truth)
        else:
            union_ground_truth = ground_truths[0]
        
        # Append the union ground truth to the list of all unions
        # union_ground_truths.append(union_ground_truth)
        union_ground_truths[mask_1] = union_ground_truth



        # # evaluate the ground truth mask with the background replacement strategy for masking function
        # predicted_fG = f_masked(np.expand_dims(ground_truth, axis=0))[0]
        # f_G = float(predicted_fG[predicted_cls])
        # print(class_names[predicted_cls], f_G, predicted_cls, f_G)
        # print('softmax prob:', np_softmax(predicted_fG)[predicted_cls])

        # # evaluate the backgrounf (negative of the ground truth mask)
        # background_mask = np.logical_not(ground_truth)
        # predicted_fB = f_masked(np.expand_dims(background_mask, axis=0))[0]
        # f_B = float(predicted_fB[predicted_cls])
        # print(class_names[predicted_cls], f_B, predicted_cls, f_B)
        # print('softmax prob:', np_softmax(predicted_fB)[predicted_cls])

        # print()
        # print('nu(S):  ', round(f_S, 4))
        # print('nu(G):  ', round(f_G, 4))
        # print('nu(S/G):', round(f_B, 4))
        # print('nu(0):  ', round(f_0, 4))

In [ ]:
# ground_truth

In [ ]:
# f_G,f_B = get_bg_values(shap_bpt_masked_model,ground_truth)

In [ ]:
run_test_anomaly_types = 'anomalous' # all
test_images_try_ls = count_cls[1:] # Discarding Non Anomolous Images

if run_test_anomaly_types=='all':
    anom_type = anomaly_types
elif run_test_anomaly_types=='anomalous':
    anom_type = anomaly_types_gt
    # test_images_try_ls = test_images_try_ls[1:]
font_size = 24 if exp_type == 'selected' else 14

print(f'{"Anomaly": <{15}} {"Image": <{15}} {"method": <{10}} {"anomaly_score": <{15}} {"max_IoU": <{15}}')
for a_type_id,a_type in enumerate(tqdm(anom_type, desc = 'anom_type')):
    for img_id in tqdm(range(0,test_images_try_ls[a_type_id]), desc ='images'):
        if a_type=='crack':
            continue
        if img_id>=3:
            continue
        fname=f'{results_path_paper_figures}/{a_type}_{img_id+1}_{exp_type}.{save_ext}'

        print('Anomaly Type: \t',a_type,' Image: \t',img_id)
        load_image(anomaly_type=a_type, img_id=img_id)
        f_0 = shap_bpt_masked_model(np.zeros((1,128,128), dtype=bool))[0,0]
        f_S = shap_bpt_masked_model(np.ones((1,128,128), dtype=bool))[0,0]
        f_G,f_B = get_bg_values(shap_bpt_masked_model,ground_truth, verbose=True)
        
        get_anomaly_map_score()
        heatmaps = {}
        for n,_,funct in tqdm(methods, desc= 'Explanation'):
            if n not in selected_methods:
                continue
            heatmaps[n] = funct()
        aucD,aucI,IoU = {},{},{}
        for n,_,funct in tqdm(methods, desc= 'Evaluation'): 
            if n not in selected_methods:
                continue
            aucD[n] = saliency_to_auc(shap_bpt_masked_model,heatmaps[n],f_S, f_0, predicted_cls=0, method='del', batch_size=batch_size)
            aucI[n] = saliency_to_auc(shap_bpt_masked_model,heatmaps[n],f_S, f_0, predicted_cls=0, method='ins', batch_size=batch_size)
            # IoU[n]  = ut.visualize.calc_IoU_curve(y_true_flat, heatmaps[n].flatten())
            IoU[n]  = calc_IoU_curve(ground_truth[:,:,0].astype('bool').flatten(), heatmaps[n].flatten())
        # plot_figures_full_filtered(heatmaps,selected_methods=selected_methods,fname=fname,plot_colorbar=False,plot_title=False,
        #                            fontsize= font_size,text_x=15 ,text_y=40)
        if exp_type == 'selected':
            plot_figures_full_filtered(heatmaps,selected_methods=selected_methods,plot_recon=False,fname=fname,plot_colorbar=False,plot_title=False,
                                  fontsize= font_size,text_x=15 ,text_y=40, destroy_fig=False)
        else:
            plot_figures_full_filtered(heatmaps,selected_methods=selected_methods,plot_recon=True,fname=fname,plot_colorbar=False,plot_title=False,
                                  fontsize= font_size,text_x=15 ,text_y=40, destroy_fig=False)
        fun_plot_performance(aucI,aucD,IoU,path=results_path, file_name=img_id,layout='2rows',
                                     set_title=True,ttl=f'a_type: {a_type}, img_id: {img_id} exp_type: {exp_type}')
        # fun_plot_performance(aucI,aucD,IoU,ext_type=save_ext, path=f'{results_path_paper_figures}/{a_type}_{img_id}_{exp_type}')

## RUN FOR SELECTED EXAMPLE

In [ ]:
a_type = 'hole'
img_id = 11

load_image(anomaly_type=a_type, img_id=img_id)
f_0 = shap_bpt_masked_model(np.zeros((1,128,128), dtype=bool))[0,0]
f_S = shap_bpt_masked_model(np.ones((1,128,128), dtype=bool))[0,0]
f_G,f_B = get_bg_values(shap_bpt_masked_model,ground_truth, verbose=True)

get_anomaly_map_score()
heatmaps = {}
for n,_,funct in tqdm(methods, desc= 'Explanation'):
    if n not in selected_methods:
        continue
    # print(f'Computing {n}')
    st = datetime.now()
    heatmaps[n] = funct()
    print(f'Computed {n:<10} in  {format(datetime.now() - st):<10}')

aucD,aucI,IoU = {},{},{}
for n,_,funct in tqdm(methods, desc= 'Evaluation'): 
    if n not in selected_methods:
        continue
    
    aucD[n] = saliency_to_auc(shap_bpt_masked_model,heatmaps[n],f_S, f_0, predicted_cls=0, method='del', batch_size=batch_size)
    aucI[n] = saliency_to_auc(shap_bpt_masked_model,heatmaps[n],f_S, f_0, predicted_cls=0, method='ins', batch_size=batch_size)
    # IoU[n]  = ut.visualize.calc_IoU_curve(y_true_flat, heatmaps[n].flatten())
    IoU[n]  = calc_IoU_curve(ground_truth[:,:,0].astype('bool').flatten(), heatmaps[n].flatten())
# plot_figures_full_filtered(heatmaps,selected_methods=selected_methods,fname=fname,plot_colorbar=False,plot_title=False,
#                            fontsize= font_size,text_x=15 ,text_y=40)
if exp_type == 'selected':
    plot_figures_full_filtered(heatmaps,selected_methods=selected_methods,plot_recon=False,fname=fname,plot_colorbar=False,plot_title=False,
                            fontsize= font_size,text_x=15 ,text_y=40, destroy_fig=False)
else:
    plot_figures_full_filtered(heatmaps,selected_methods=selected_methods,plot_recon=True,fname=fname,plot_colorbar=False,plot_title=False,
                            fontsize= font_size,text_x=15 ,text_y=40, destroy_fig=False)
fun_plot_performance(aucI,aucD,IoU,path=results_path, file_name=img_id,layout='2rows',
                                set_title=True,ttl=f'a_type: {a_type}, img_id: {img_id} exp_type: {exp_type}')

# Run Full Test

In [ ]:
run_full_set = True
# run_full_set = False

exp_name = 'csv_exp_E6'

In [ ]:
importlib.reload(ut)
if run_full_set:
    start_time = datetime.now()

    plot_IoU            = True
    save_figs           = True
    destroy_fig         = True
    plot_summary        = False
    verbose             = True
    verbose_print       = False
    num_samples         = 500
    batch_size          = 200
    target_segs         = 100
    fontsize            = 14
    mask_types          = ['blend_2']
    postfix_csv = f'{DS_name}_{len(methods)}'
    csv_filename = f'{path_csv}/{exp_name}_testresults_{postfix_csv}_BPT_new_eval.csv'
    print('csv_filename: ', csv_filename)
    print('anomaly types in test set:\t',anomaly_types_gt)
    test_images_try_ls = count_cls[1:] # Discarding Non Anomolous Images
    props = dict(boxstyle='round', facecolor='white', alpha=0.6, pad=0.2)
    total_iterations = len(methods) * sum(test_images_try_ls)
    pbar = tqdm(desc=f"Progress", total=total_iterations)
    st_full       = time.time()

    run_test_anomaly_types = 'anomalous' # all
    if run_test_anomaly_types=='all':
        anom_type = anomaly_types
    elif run_test_anomaly_types=='anomalous':
        anom_type = anomaly_types_gt
        # test_images_try_ls = test_images_try_ls[1:]
    if verbose_print:    
        print(f'{"Anomaly": <{15}} {"Image": <{15}} {"method": <{10}} {"anomaly_score": <{15}} {"max_IoU": <{15}}')
    results=[]
    for a_type_id,a_type in enumerate(anom_type):
        for img_id in range(0,test_images_try_ls[a_type_id]):
            
            fig,axes = plt.subplots(1, len(methods)+4, figsize=(15, 3))
            
            time_start = time.time()
            results_subpath_cls = os.path.join(results_subs,str(a_type))
            os.makedirs(results_subpath_cls,               exist_ok=True)
            st_load = time.time()
            load_image(anomaly_type=a_type, img_id=img_id)
            get_anomaly_map_score()
            f_0 = shap_bpt_masked_model(np.zeros((1,128,128), dtype=bool))[0,0]
            f_S = shap_bpt_masked_model(np.ones((1,128,128), dtype=bool))[0,0]

            f_G,f_B = get_bg_values(shap_bpt_masked_model,ground_truth)

            anomaly_score_val = score_function(np.array([I_A]))[0][0]
            time_load = time.time()-st_load
            
            AM_roc_score      = 0.0 if a_type=='good' else sklearn.metrics.roc_auc_score(y_true_flat, predicted_flat)
            
            axes[0].imshow(I_A)
            axes[1].imshow(R_A)
            axes[2].imshow(anomaly_map, vmin=-am_max, vmax=am_max, cmap = shap_bpt.shapley_values_colormap)
            contoured_image = draw_plot_gt(I_A,ground_truth)
            axes[-1].imshow(contoured_image)
            for mask_type_id,mask_type in enumerate(mask_types):
                get_mask(mask_type=mask_type)
                mask_ttl = f'{mask_tp},k:{num_samples}'
                heatmaps,aucD,aucI,IoU = {},{},{},{}
                time_exp = {}
                for n,_,funct in tqdm(methods, leave=False, desc = 'Explanations'):
                    data={}
                    st = time.time()
                    heatmaps[n] = funct()
                    time_exp[n] = time.time()-st
                    #################    aucD
                    st = time.time()
                    aucD[n] = saliency_to_auc(shap_bpt_masked_model,heatmaps[n],f_S, f_0, predicted_cls=0, method='del', batch_size=batch_size)
            
                    time_aucD = time.time()-st
                    #################    aucI
                    st = time.time()
                    aucI[n] = saliency_to_auc(shap_bpt_masked_model,heatmaps[n],f_S, f_0, predicted_cls=0, method='ins', batch_size=batch_size)
            
                    time_aucI = time.time()-st
                    if n=='LIME':
                        cv_beta = ut.Explanation.get_CV_beta(beta)
                        auroc       = 0.0 if a_type=='good' else sklearn.metrics.roc_auc_score(y_true_flat, heatmaps[n].flatten())
                        LIME_roc_abs_score    = 0.0 if a_type=='good' else sklearn.metrics.roc_auc_score(y_true_flat, np.abs(heatmaps[n].flatten()))
                        
                        heatmap_gs1 = gaussian_filter(heatmap_lime, sigma=1)
                        heatmap_gs2 = gaussian_filter(heatmap_lime, sigma=2)

                    else:
                        auroc,auroc_gs1,auroc_gs2 = 0,0,0
                    # IoU[n] =ut.visualize.calc_IoU_curve(y_true_flat, heatmaps[n].flatten())
                    # IoU[n]  = ut.visualize.calc_IoU_curve(y_true_flat, heatmaps[n].flatten())
                    st = time.time()
                    IoU[n]  = calc_IoU_curve(ground_truth[:,:,0].astype('bool').flatten(), heatmaps[n].flatten())
                    time_IoU = time.time()-st
                    if verbose_print:
                        print(f"{a_type: <{15}} {img_id: <{15}} {n: <{10}} {am_max: <{15.4}} {np.max(IoU[n]['Y']): <{15.4}}")
                    data = {
                            'a_type_id'         :  a_type_id,
                            'a_type'            :  a_type,
                            'img_no'            :  img_id,
                            'mask_type'         :  mask_type,
                            'target_segs'       :  target_segs,
                            'num_samples'       :  num_samples,
                            #------------------------------------------------------------------------------------------------#
                            'anomaly_score'     :  am_max,
                            'anomaly_score_val' :  anomaly_score_val,
                            'AM_roc_score'      :  AM_roc_score,
                            'auroc'             :  auroc,
                            #--------------------------------
                            'max_heatmap'       :  np.max(heatmaps[n]),
                            #------------------------------------------------------------------------------------------------#
                            'f_S'               : f_S,
                            'f_0'               : f_0,
                            'f_G'               : f_B,
                            'f_B'               : f_G,
                            'delta_f'           : f_S-f_0,
                            'f_T'               : np.sum(heatmaps[n][0]),
                            'f_N'               : len(np.unique(heatmaps[n][0])),           ## UNIQUE PATCHES IN EXPLANATION
                            #------------------------------------------------------------------------------------------------#
                            'method'            :  n,
                            #--------------------------------
                            'aucI_pred'         : aucI[n]['auc'],
                            'aucD_pred'         : aucD[n]['auc'],
                            #--------------------------------
                            'aucI_r'            : aucI[n]['auc_r'],
                            'aucD_r'            : aucD[n]['auc_r'],
                            #--------------------------------
                            'aucI_adj'          : aucI[n]['auc_adj'],
                            'aucD_adj'          : aucD[n]['auc_adj'],
                            #--------------------------------
                            'aucI_adj_r'        : aucI[n]['auc_adjr'],
                            'aucD_adj_r'        : aucD[n]['auc_adjr'],
                            #--------------------------------
                            'aucI_clip'         : aucI[n]['auc_clip'], 
                            'aucI_clipr'        : aucI[n]['auc_clipr'],
                            #--------------------------------
                            'aucD_clip'         : aucD[n]['auc_clip'], 
                            'aucD_clipr'        : aucD[n]['auc_clipr'],
                            #--------------------------------
                            'threshold'         : IoU[n][2],
                            'best_point'        : IoU[n][3],
                            'max_IoU'           : np.max(IoU[n][1]),
                            'au_IoU'            : IoU[n][4],
                            #------------------------------------------------------------------------------------------------#
                            'time_exp'          : time_exp[n],
                            'time_aucI'         : time_aucI,
                            'time_aucD'         : time_aucD,
                            'time_IoU'          : time_IoU,
                            'time_total'        : time_load+time_exp[n]+time_aucI+time_aucD+time_IoU,
                            #------------------------------------------------------------------------------------------------#
                            
                            }
                    results.append(data)
                    df = pd.DataFrame(results)
                    df.to_csv(csv_filename, sep=',')
                    pbar.update(1)
                if verbose_print:
                    print('-'*120)
                for n_id,(n,_,funct) in enumerate(methods):
                    ax = axes[n_id+3]
                    maxval = np.max(np.abs(heatmaps[n]))
                    heatmap_clr_bar = ax.imshow(heatmaps[n], cmap=shap_bpt.shapley_values_colormap, vmin=-maxval, vmax=maxval)
                    marked_h = mark_boundaries(np.tile((255,255,255,0), (heatmaps[n].shape[0],heatmaps[n].shape[1],1)), ground_truth[:,:,0], 
                             mode='thick', color=(0,0,0,1))
                    ax.imshow(marked_h)
                for ax in axes.flatten(): ax.set_xticks([]) ; ax.set_yticks([])
                plt.subplots_adjust(wspace=0.05,hspace=0.05)
                if save_figs:
                    # plt.savefig(f'{results_subpath_cls}/{a_type}_{img_id}_{epochs}_{mask_type}.{selected_ext}',dpi=250, transparent=True, bbox_inches = 'tight')
                    plt.savefig(f'{results_subpath_cls}/{img_id}_heatmaps_{a_type}_{img_id}_{epochs}_{mask_type}.png',dpi=150, transparent=True, bbox_inches = 'tight')
                if destroy_fig:
                        plt.close(fig)
                plt.show()
                if plot_IoU:
                    fig,axes = plt.subplots(1, len(methods)+4, figsize=(15, 3))
                    axes[0].imshow(I_A)
                    axes[1].imshow(R_A)
                    axes[2].imshow(anomaly_map, vmin=-am_max, vmax=am_max, cmap = shap_bpt.shapley_values_colormap)
                    axes[-1].imshow(contoured_image)
                    for n_id,(n,_,funct) in enumerate(methods):
                        ax = axes[n_id+3]
                        max___iou = IoU[n][2]
                        IoU__Y =  np.max(IoU[n][1])
                        img, max_IoU, name = vis_IoU(heatmaps[n],max___iou , ground_truth[:, :, 0] != 0), IoU__Y, f'{n}'
                        # img, max_IoU, name = ut.visualize.vis_IoU(heatmaps[n], IoU[n]['max_IoU_old'],ground_truth[:,:,0]!=0),np.max(IoU[n]['Y']), f'{n}'
                        
                        ax.imshow(img)
                        ax.text(10,30, f"IoU:{max_IoU:.3}", fontsize=fontsize-2, bbox=props,weight='bold')
                    
                    for ax in axes.flatten():  ax.set_xticks([]) ; ax.set_yticks([])
                    plt.subplots_adjust(wspace=0.05,hspace=0.05)
                    if save_figs:
                        # plt.savefig(f'{results_subpath_cls}/{a_type}_{img_id}_{epochs}_{mask_type}.{selected_ext}',dpi=250, transparent=True, bbox_inches = 'tight')
                        plt.savefig(f'{results_subpath_cls}/{img_id}_IoU_{a_type}_{img_id}_{epochs}_{mask_type}.png',dpi=250, transparent=True, bbox_inches = 'tight')
                    if destroy_fig:
                        plt.close(fig)
                    plt.show()
    time_full = time.time()-st_full
    end_time = datetime.now()
    print(f"Time for test is :, {time_full}\n")
    print(f"Duration: {format(end_time - start_time)}")

In [ ]:
print(f"Duration: {format(end_time - start_time)}")
print(f'FILE SAVED AT {csv_filename}')

In [ ]:
# if run_full_set:
print(df.method.unique())
df.head(3)

# END